In [ ]:

### import all the dependencies 
import requests
import csv
import json
import pandas as pd
import requests, json
import re
import configparser

In [ ]:
## Define all variables 
configFile = "config.cfg"
config = configparser.ConfigParser()
config.read(configFile)


In [ ]:
### Main Program starts here  - Now we will read data form CSV

customers_df = pd.read_csv("../data/Maindata.csv", low_memory=False)
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

In [ ]:
##List the names of columns 
customers_df.columns


In [ ]:
len(customers_df.columns)

In [ ]:
## Data Cleaning and filtering 
#dropping an Column not needed 
print('number of columns  before :{}'.format(len(customers_df.columns)))
customers_df.drop('Unnamed: 0', axis=1, inplace=True)
print('number of columns  After :{}'.format(len(customers_df.columns)))

In [ ]:
## Need to Add a column as Domain 
print('Adding domain column')

customers_df['Domain'] = customers_df['UserEmail'].str.split(pat="@", expand=True)[1]
print('Shape of exports read from CSV : {}'.format(customers_df.shape))

In [ ]:

# drop rows that doesn't have an email
customers_df = customers_df.dropna(subset=['UserEmail'])

# remove all the rows that use internal email or not valid email
#Read the info on emails fron config file 
dropemails= config.get('Emails', 'dropEmailList')
dropemailsList = dropemails.split(",")
external_customers_df = customers_df[~ customers_df.Domain.isin(dropemailsList)]

print('Shape of dataframe after dropping internal users : {}'.format(external_customers_df.shape))



In [ ]:
#quick preview of data
external_customers_df.head()


In [ ]:
## Dropping unwanted columns 
external_customers_df = external_customers_df[['Domain','UserAppKey','user_serviceName_str','UserId','IndvId','SessionId',
                                       'PageUrl','PageRefererUrl','PageBrowser','PageDevice','PageDuration','PageActiveDuration',
                                        'PageLatLong',
                                       'EventStart','EventType']]
print('Shape of dataframe after dropping extra columns: {}'.format(external_customers_df.shape))                                    

In [ ]:
##Renaming columns
external_customers_df.rename(columns={'UserAppKey':'UserEmail','user_serviceName_str':'serviceName'},inplace = True)
external_customers_df.columns

In [ ]:
## Printing list of unique services we have collected data for 
list(external_customers_df.serviceName.unique())

In [ ]:
## Dropping rows with extra services we dont care they were part of some POC 
##servciesName = list(customers_data.user_serviceName_str.unique())
print("Size of data before dropping servcies {}".format(external_customers_df.shape))
service_picked= ['LOG_INTELLIGENCE','CloudAssembly','SDK']
external_customers_df = external_customers_df[external_customers_df.serviceName.isin(service_picked)]
print("Size of data after  dropping servcies {} ".format(external_customers_df.shape))

In [ ]:
## REPLACING SERVICES NAMES to make them more meaning full
external_customers_df.replace("LOG_INTELLIGENCE","Data Logging",inplace =True)
external_customers_df.replace('CloudAssembly',"Data Package",inplace =True)
print("Size of data {}".format(external_customers_df.shape))
list(external_customers_df.serviceName.unique())

In [ ]:
#Masking Company Names 
domainList  = list(external_customers_df.Domain.unique())
counter = 1
for domain in domainList  :
    newDomain = "Company" + str(counter)+".com"
    print(newDomain)
    external_customers_df.replace(domain,newDomain,inplace =True,regex=True)
    counter=counter +1  

print("Masking done")
external_customers_df.head(20)

In [ ]:
#Masking Some Sensitive URL
external_customers_df.replace("/www.mgmt.cloud.vmware.com/","/www.cloud.dataguru.com/",inplace =True,regex=True)
external_customers_df.replace("vmware","dataguru",inplace =True,regex=True)
external_customers_df.replace("csp","bin",inplace =True,regex=True)
external_customers_df.replace("deployment","dataType",inplace =True,regex=True)
external_customers_df.replace("automation-ui","dataType-UI",inplace =True,regex=True)
external_customers_df.replace("provisioning","data-provision",inplace =True,regex=True)


In [ ]:
### Writing  masked /filtered and clean data to CSV ,from here always read the local file names external_customers.csv
external_customers_df.to_csv("../data/external_customers.csv")
print("external Customer data  Successfully written to external_customers.csv ")

In [ ]:
## Now We need to answer active users or company by unique  session ID count 
##This is sample code anyway we will do the processing in another file 
df_groupby_session = external_customers_df.groupby(["Domain",'UserEmail'])['UserId'].nunique()
df_groupby_session


In [ ]:
## Now We need to answer most used service 

df_groupby_service = external_customers_df.groupby(["serviceName"])["PageUrl"].count()
df_groupby_service